In [1]:
print('22K-4637')

22K-4637


In [ ]:
# variables : Each step in the path
# domains : Diagonal Negihbours i.e (x-1,y-1) , (x-1,y+1) , (x+1,y-1) , (x+1,y-1)
# constraints : 
                 # 1 No obstacle constraint
                 # 2 Valid move constraint
                 # 3 Path continuity



In [2]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import math

def create_data_model():
    data = {}
    grid_size = 5
    start = (1, 1)
    end = (4, 4)
    obstacles = set([
        (2, 2), (3, 3)
    ])

    nodes = []
    pos_to_idx = {}
    idx_to_pos = {}
    index = 0
    for x in range(grid_size):
        for y in range(grid_size):
            if (x, y) not in obstacles:
                pos_to_idx[(x, y)] = index
                idx_to_pos[index] = (x, y)
                nodes.append((x, y))
                index += 1

    size = len(nodes)
    dist_matrix = [[999999 for _ in range(size)] for _ in range(size)]
    for i in range(size):
        for j in range(size):
            x1, y1 = nodes[i]
            x2, y2 = nodes[j]
            if abs(x1 - x2) == 1 and abs(y1 - y2) == 1:
                dist_matrix[i][j] = math.sqrt(2)

    data['distance_matrix'] = dist_matrix
    data['start_index'] = pos_to_idx[start]
    data['end_index'] = pos_to_idx[end]
    data['num_nodes'] = len(nodes)
    data['idx_to_pos'] = idx_to_pos
    return data

def solve_diagonal_path():
    data = create_data_model()

    manager = pywrapcp.RoutingIndexManager(
    data['num_nodes'], 1, [data['start_index']], [data['end_index']]
    )

    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return int(data['distance_matrix'][from_node][to_node] * 1000)  # scale to int

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    routing.AddDisjunction([i for i in range(data['num_nodes'])], 999999)

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        index = routing.Start(0)
        path = []
        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            path.append(data['idx_to_pos'][node])
            index = solution.Value(routing.NextVar(index))
        path.append(data['idx_to_pos'][manager.IndexToNode(index)])
        return path
    else:
        return None


load C:\Users\ha159\AppData\Local\Programs\Python\Python312\Lib\site-packages\ortools\.libs\zlib1.dll...
load C:\Users\ha159\AppData\Local\Programs\Python\Python312\Lib\site-packages\ortools\.libs\abseil_dll.dll...
load C:\Users\ha159\AppData\Local\Programs\Python\Python312\Lib\site-packages\ortools\.libs\utf8_validity.dll...
load C:\Users\ha159\AppData\Local\Programs\Python\Python312\Lib\site-packages\ortools\.libs\re2.dll...
load C:\Users\ha159\AppData\Local\Programs\Python\Python312\Lib\site-packages\ortools\.libs\libprotobuf.dll...
load C:\Users\ha159\AppData\Local\Programs\Python\Python312\Lib\site-packages\ortools\.libs\highs.dll...
load C:\Users\ha159\AppData\Local\Programs\Python\Python312\Lib\site-packages\ortools\.libs\ortools.dll...


In [ ]:
path = solve_diagonal_path()
if path:
    print("Shortest Diagonal Path:")
    for step in path:
        print(step)
else:
    print("No valid path found.")
